In [ ]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle

from code_source.dataframe_prep import get_cols_names
from code_source import data_prep
from code_source import metrics
from code_source import learning

# dataframe preparation

In [ ]:
data=pd.read_csv('data/Dataset/Dataset/Training/Features_Variant_1.csv', header=None).drop_duplicates()
data=get_cols_names(data)
data=data.drop(columns=['post_promotion']) #post promotion has only zero values we can drop this col because it isn't informative

# data preparation

In [ ]:
#'page_cat' to dummy because it is a category
#data=pd.concat([data,pd.get_dummies(data['page_cat'], prefix='page_cat')], axis=1)
#data=data.drop(columns=['page_cat'])

In [ ]:
#data shuffling
data=shuffle(data).reset_index(drop=True)

# learning

In [ ]:
#set algo params:
lam=1e-4
gamma=0 #now we needn't it
terms_num=100
max_iter=50

# Cross-val

In [ ]:
data.iloc[:,:-1], mean, std =data_prep.normalize(data.iloc[:,:-1])

res=pd.DataFrame()

parts=range(1,6)
idxs=[round(data.shape[0]/5)*(i-1) for i in range(1,6)]
idxs.append(data.shape[0])

for part in parts:
    #separate dataset
    test_part=data[idxs[part-1]:idxs[part]]
    train_part=data.drop(data.index[idxs[part-1]:idxs[part]])

    #features normalization (xi-mean)/std_err:
    #train_part.iloc[:,:-1], mean, std=normalize(train_part.iloc[:,:-1])#fit and transform train
    #test_part.iloc[:,:-1]=normalize_with_params(test_part.iloc[:,:-1], mean, std) #transform test
    
    params=learning.gradient_descend(train_part, lam, gamma, terms_num, max_iter) #model    
    test_pred=learning.get_prediction(params[:-1],params[-1],test_part.iloc[:,:-1])
    RMSE=metrics.custom_RMSE(test_part['target'], test_pred, gamma, params[:-1])
    r2=metrics.custom_R2(test_part['target'], test_pred)
    params=np.insert(params, 0 , [RMSE,r2])
    res[part]=params

In [ ]:
text_idxs=np.array(['w'+str(i) for i in range(1,res.shape[0]-2)])
text_idxs=np.append(text_idxs,'w0')
text_idxs=np.insert(text_idxs, 0, ['RMSE', 'r2'])
res.index=text_idxs

res['mean']=np.mean(res, axis=1)
res['std']=np.std(res.iloc[:,:-1], axis=1)

In [ ]:
res